# Media Storage

> Standardized SQLite storage for media analysis and processing results with content hashing

In [ ]:
#| default_exp storage

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
import sqlite3
import time
from dataclasses import dataclass
from typing import Any, Dict, List, Optional

from cjm_plugin_system.utils.hashing import hash_file

## MediaAnalysisRow

A dataclass representing a single row in the standardized `analysis_jobs` table.

In [ ]:
#| export
@dataclass
class MediaAnalysisRow:
    """A single row from the analysis_jobs table."""
    file_path: str       # Path to the analyzed media file
    file_hash: str       # Hash of source file in "algo:hexdigest" format
    config_hash: str     # Hash of the analysis config used
    ranges: Optional[List[Dict[str, Any]]] = None  # Detected temporal segments
    metadata: Optional[Dict[str, Any]] = None       # Analysis metadata
    created_at: Optional[float] = None               # Unix timestamp

In [ ]:
# Test MediaAnalysisRow creation
row = MediaAnalysisRow(
    file_path="/tmp/test.mp3",
    file_hash="sha256:" + "a" * 64,
    config_hash="sha256:" + "b" * 64,
    ranges=[{"start": 0.0, "end": 2.5, "label": "speech"}],
    metadata={"segment_count": 1}
)

print(f"Row: file_path={row.file_path}")
print(f"File hash: {row.file_hash[:20]}...")
print(f"Config hash: {row.config_hash[:20]}...")

Row: file_path=/tmp/test.mp3
File hash: sha256:aaaaaaaaaaaaa...
Config hash: sha256:bbbbbbbbbbbbb...


## MediaAnalysisStorage

Standardized SQLite storage that all media analysis plugins should use. Defines the canonical schema for the `analysis_jobs` table with file hashing for traceability and config-based caching.

**Schema:**

```sql
CREATE TABLE IF NOT EXISTS analysis_jobs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    file_path TEXT NOT NULL,
    file_hash TEXT NOT NULL,
    config_hash TEXT NOT NULL,
    ranges JSON,
    metadata JSON,
    created_at REAL NOT NULL,
    UNIQUE(file_path, config_hash)
);
```

The `UNIQUE(file_path, config_hash)` constraint enables result caching — re-running the same file with the same config replaces the previous result. Different configs for the same file are stored separately.

In [ ]:
#| export
class MediaAnalysisStorage:
    """Standardized SQLite storage for media analysis results."""

    SCHEMA = """
        CREATE TABLE IF NOT EXISTS analysis_jobs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            file_path TEXT NOT NULL,
            file_hash TEXT NOT NULL,
            config_hash TEXT NOT NULL,
            ranges JSON,
            metadata JSON,
            created_at REAL NOT NULL,
            UNIQUE(file_path, config_hash)
        )
    """

    INDEX = "CREATE INDEX IF NOT EXISTS idx_analysis_file_path ON analysis_jobs(file_path);"

    def __init__(
        self,
        db_path: str  # Absolute path to the SQLite database file
    ):
        """Initialize storage and create table if needed."""
        self.db_path = db_path
        with sqlite3.connect(self.db_path) as con:
            con.execute(self.SCHEMA)
            con.execute(self.INDEX)

    def save(
        self,
        file_path: str,     # Path to the analyzed media file
        file_hash: str,     # Hash of source file in "algo:hexdigest" format
        config_hash: str,   # Hash of the analysis config
        ranges: Optional[List[Dict[str, Any]]] = None,  # Detected temporal segments
        metadata: Optional[Dict[str, Any]] = None        # Analysis metadata
    ) -> None:
        """Save or replace an analysis result (upsert by file_path + config_hash)."""
        with sqlite3.connect(self.db_path) as con:
            con.execute(
                """INSERT OR REPLACE INTO analysis_jobs
                   (file_path, file_hash, config_hash, ranges, metadata, created_at)
                   VALUES (?, ?, ?, ?, ?, ?)""",
                (
                    file_path,
                    file_hash,
                    config_hash,
                    json.dumps(ranges) if ranges else None,
                    json.dumps(metadata) if metadata else None,
                    time.time()
                )
            )

    def get_cached(
        self,
        file_path: str,   # Path to the media file
        config_hash: str  # Config hash to match
    ) -> Optional[MediaAnalysisRow]:  # Cached row or None
        """Retrieve a cached analysis result by file path and config hash."""
        with sqlite3.connect(self.db_path) as con:
            cur = con.execute(
                """SELECT file_path, file_hash, config_hash, ranges, metadata, created_at
                   FROM analysis_jobs WHERE file_path = ? AND config_hash = ?""",
                (file_path, config_hash)
            )
            row = cur.fetchone()
            if not row:
                return None
            return MediaAnalysisRow(
                file_path=row[0],
                file_hash=row[1],
                config_hash=row[2],
                ranges=json.loads(row[3]) if row[3] else None,
                metadata=json.loads(row[4]) if row[4] else None,
                created_at=row[5]
            )

    def list_jobs(
        self,
        limit: int = 100  # Maximum number of rows to return
    ) -> List[MediaAnalysisRow]:  # List of analysis rows
        """List analysis jobs ordered by creation time (newest first)."""
        results = []
        with sqlite3.connect(self.db_path) as con:
            cur = con.execute(
                """SELECT file_path, file_hash, config_hash, ranges, metadata, created_at
                   FROM analysis_jobs ORDER BY created_at DESC LIMIT ?""",
                (limit,)
            )
            for row in cur:
                results.append(MediaAnalysisRow(
                    file_path=row[0],
                    file_hash=row[1],
                    config_hash=row[2],
                    ranges=json.loads(row[3]) if row[3] else None,
                    metadata=json.loads(row[4]) if row[4] else None,
                    created_at=row[5]
                ))
        return results

    def verify_file(
        self,
        file_path: str,   # Path to the media file
        config_hash: str  # Config hash to look up
    ) -> Optional[bool]:  # True if file matches, False if changed, None if not found
        """Verify the source media file still matches its stored hash."""
        row = self.get_cached(file_path, config_hash)
        if not row:
            return None
        current_hash = hash_file(row.file_path)
        return current_hash == row.file_hash

## Testing

In [ ]:
import tempfile
import os

# Create storage with temp database
tmp_db = tempfile.NamedTemporaryFile(suffix=".db", delete=False)
storage = MediaAnalysisStorage(tmp_db.name)

print(f"Storage initialized at: {tmp_db.name}")

Storage initialized at: /tmp/tmp04637ejq.db


In [ ]:
# Save an analysis result
storage.save(
    file_path="/tmp/test_audio.mp3",
    file_hash="sha256:" + "a" * 64,
    config_hash="sha256:" + "c" * 64,
    ranges=[
        {"start": 0.0, "end": 2.5, "label": "speech", "confidence": 0.98},
        {"start": 4.0, "end": 8.5, "label": "speech", "confidence": 0.95}
    ],
    metadata={"segment_count": 2, "total_speech": 7.0}
)

print("Saved analysis result")

Saved analysis result


In [ ]:
# Retrieve cached result
cached = storage.get_cached("/tmp/test_audio.mp3", "sha256:" + "c" * 64)
assert cached is not None
assert cached.file_path == "/tmp/test_audio.mp3"
assert len(cached.ranges) == 2
assert cached.metadata["segment_count"] == 2
assert cached.created_at is not None

print(f"Cached: {cached.file_path}")
print(f"Ranges: {len(cached.ranges)} segments")
print(f"File hash: {cached.file_hash[:20]}...")

# Missing config returns None
missing = storage.get_cached("/tmp/test_audio.mp3", "sha256:" + "d" * 64)
assert missing is None
print("Cache miss for different config: OK")

Cached: /tmp/test_audio.mp3
Ranges: 2 segments
File hash: sha256:aaaaaaaaaaaaa...
Cache miss for different config: OK


In [ ]:
# Save with same file+config replaces (upsert)
storage.save(
    file_path="/tmp/test_audio.mp3",
    file_hash="sha256:" + "a" * 64,
    config_hash="sha256:" + "c" * 64,
    ranges=[{"start": 0.0, "end": 3.0, "label": "speech"}],
    metadata={"segment_count": 1, "total_speech": 3.0}
)

updated = storage.get_cached("/tmp/test_audio.mp3", "sha256:" + "c" * 64)
assert len(updated.ranges) == 1  # Updated to 1 range
assert updated.metadata["segment_count"] == 1

# Only 1 row total (replaced, not appended)
all_jobs = storage.list_jobs()
assert len(all_jobs) == 1

print("Upsert replaced existing row: OK")

Upsert replaced existing row: OK


In [ ]:
# Different config for same file creates separate row
storage.save(
    file_path="/tmp/test_audio.mp3",
    file_hash="sha256:" + "a" * 64,
    config_hash="sha256:" + "e" * 64,  # Different config
    ranges=[{"start": 0.5, "end": 2.0, "label": "speech"}],
    metadata={"segment_count": 1}
)

all_jobs = storage.list_jobs()
assert len(all_jobs) == 2

print(f"Two configs for same file: {len(all_jobs)} rows")

Two configs for same file: 2 rows


In [ ]:
# Cleanup
os.unlink(tmp_db.name)
print("Cleanup complete")

Cleanup complete


## MediaProcessingRow

A dataclass representing a single row in the standardized `processing_jobs` table. Tracks input/output file pairs with hashes for full traceability of media transformations.

In [ ]:
#| export
@dataclass
class MediaProcessingRow:
    """A single row from the processing_jobs table."""
    job_id: str          # Unique job identifier
    action: str          # Operation performed: 'convert', 'extract_segment', etc.
    input_path: str      # Path to the source media file
    input_hash: str      # Hash of source file in "algo:hexdigest" format
    output_path: str     # Path to the produced output file
    output_hash: str     # Hash of output file in "algo:hexdigest" format
    parameters: Optional[Dict[str, Any]] = None  # Action-specific parameters
    metadata: Optional[Dict[str, Any]] = None     # Processing metadata
    created_at: Optional[float] = None             # Unix timestamp

In [ ]:
# Test MediaProcessingRow creation
proc_row = MediaProcessingRow(
    job_id="job_conv_001",
    action="convert",
    input_path="/tmp/source.mkv",
    input_hash="sha256:" + "a" * 64,
    output_path="/tmp/output.mp4",
    output_hash="sha256:" + "b" * 64,
    parameters={"output_format": "mp4", "codec": "h264"}
)

print(f"Row: job_id={proc_row.job_id}, action={proc_row.action}")
print(f"Input: {proc_row.input_path} -> Output: {proc_row.output_path}")

Row: job_id=job_conv_001, action=convert
Input: /tmp/source.mkv -> Output: /tmp/output.mp4


## MediaProcessingStorage

Standardized SQLite storage that all media processing plugins should use. Defines the canonical schema for the `processing_jobs` table, tracking input/output file pairs with content hashes.

**Schema:**

```sql
CREATE TABLE IF NOT EXISTS processing_jobs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    job_id TEXT UNIQUE NOT NULL,
    action TEXT NOT NULL,
    input_path TEXT NOT NULL,
    input_hash TEXT NOT NULL,
    output_path TEXT NOT NULL,
    output_hash TEXT NOT NULL,
    parameters JSON,
    metadata JSON,
    created_at REAL NOT NULL
);
```

Both `input_hash` and `output_hash` use the self-describing `"algo:hexdigest"` format, enabling verification of both source integrity ("is this the same file we converted?") and output integrity ("has the output been modified since conversion?").

In [ ]:
#| export
class MediaProcessingStorage:
    """Standardized SQLite storage for media processing results."""

    SCHEMA = """
        CREATE TABLE IF NOT EXISTS processing_jobs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            job_id TEXT UNIQUE NOT NULL,
            action TEXT NOT NULL,
            input_path TEXT NOT NULL,
            input_hash TEXT NOT NULL,
            output_path TEXT NOT NULL,
            output_hash TEXT NOT NULL,
            parameters JSON,
            metadata JSON,
            created_at REAL NOT NULL
        )
    """

    INDEX = "CREATE INDEX IF NOT EXISTS idx_processing_job_id ON processing_jobs(job_id);"

    def __init__(
        self,
        db_path: str  # Absolute path to the SQLite database file
    ):
        """Initialize storage and create table if needed."""
        self.db_path = db_path
        with sqlite3.connect(self.db_path) as con:
            con.execute(self.SCHEMA)
            con.execute(self.INDEX)

    def save(
        self,
        job_id: str,        # Unique job identifier
        action: str,        # Operation performed: 'convert', 'extract_segment', etc.
        input_path: str,    # Path to the source media file
        input_hash: str,    # Hash of source file in "algo:hexdigest" format
        output_path: str,   # Path to the produced output file
        output_hash: str,   # Hash of output file in "algo:hexdigest" format
        parameters: Optional[Dict[str, Any]] = None,  # Action-specific parameters
        metadata: Optional[Dict[str, Any]] = None       # Processing metadata
    ) -> None:
        """Save a media processing result to the database."""
        with sqlite3.connect(self.db_path) as con:
            con.execute(
                """INSERT INTO processing_jobs
                   (job_id, action, input_path, input_hash, output_path, output_hash,
                    parameters, metadata, created_at)
                   VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)""",
                (
                    job_id,
                    action,
                    input_path,
                    input_hash,
                    output_path,
                    output_hash,
                    json.dumps(parameters) if parameters else None,
                    json.dumps(metadata) if metadata else None,
                    time.time()
                )
            )

    def get_by_job_id(
        self,
        job_id: str  # Job identifier to look up
    ) -> Optional[MediaProcessingRow]:  # Row or None if not found
        """Retrieve a processing result by job ID."""
        with sqlite3.connect(self.db_path) as con:
            cur = con.execute(
                """SELECT job_id, action, input_path, input_hash, output_path, output_hash,
                          parameters, metadata, created_at
                   FROM processing_jobs WHERE job_id = ?""",
                (job_id,)
            )
            row = cur.fetchone()
            if not row:
                return None
            return MediaProcessingRow(
                job_id=row[0],
                action=row[1],
                input_path=row[2],
                input_hash=row[3],
                output_path=row[4],
                output_hash=row[5],
                parameters=json.loads(row[6]) if row[6] else None,
                metadata=json.loads(row[7]) if row[7] else None,
                created_at=row[8]
            )

    def list_jobs(
        self,
        limit: int = 100  # Maximum number of rows to return
    ) -> List[MediaProcessingRow]:  # List of processing rows
        """List processing jobs ordered by creation time (newest first)."""
        results = []
        with sqlite3.connect(self.db_path) as con:
            cur = con.execute(
                """SELECT job_id, action, input_path, input_hash, output_path, output_hash,
                          parameters, metadata, created_at
                   FROM processing_jobs ORDER BY created_at DESC LIMIT ?""",
                (limit,)
            )
            for row in cur:
                results.append(MediaProcessingRow(
                    job_id=row[0],
                    action=row[1],
                    input_path=row[2],
                    input_hash=row[3],
                    output_path=row[4],
                    output_hash=row[5],
                    parameters=json.loads(row[6]) if row[6] else None,
                    metadata=json.loads(row[7]) if row[7] else None,
                    created_at=row[8]
                ))
        return results

    def verify_input(
        self,
        job_id: str  # Job identifier to verify
    ) -> Optional[bool]:  # True if input matches, False if changed, None if not found
        """Verify the source media file still matches its stored hash."""
        row = self.get_by_job_id(job_id)
        if not row:
            return None
        current_hash = hash_file(row.input_path)
        return current_hash == row.input_hash

    def verify_output(
        self,
        job_id: str  # Job identifier to verify
    ) -> Optional[bool]:  # True if output matches, False if changed, None if not found
        """Verify the output media file still matches its stored hash."""
        row = self.get_by_job_id(job_id)
        if not row:
            return None
        current_hash = hash_file(row.output_path)
        return current_hash == row.output_hash

### Testing MediaProcessingStorage

In [ ]:
# Create processing storage with temp database
tmp_db2 = tempfile.NamedTemporaryFile(suffix=".db", delete=False)
proc_storage = MediaProcessingStorage(tmp_db2.name)

print(f"Processing storage initialized at: {tmp_db2.name}")

Processing storage initialized at: /tmp/tmp6cy8gfmq.db


In [ ]:
# Save a conversion job
proc_storage.save(
    job_id="job_conv_001",
    action="convert",
    input_path="/tmp/source.mkv",
    input_hash="sha256:" + "a" * 64,
    output_path="/tmp/output.mp4",
    output_hash="sha256:" + "b" * 64,
    parameters={"output_format": "mp4", "codec": "h264"},
    metadata={"duration": 120.5}
)

print("Saved conversion job")

Saved conversion job


In [ ]:
# Retrieve by job ID
row = proc_storage.get_by_job_id("job_conv_001")
assert row is not None
assert row.job_id == "job_conv_001"
assert row.action == "convert"
assert row.input_path == "/tmp/source.mkv"
assert row.output_path == "/tmp/output.mp4"
assert row.parameters["output_format"] == "mp4"
assert row.created_at is not None

print(f"Retrieved: {row.job_id} ({row.action})")
print(f"Input: {row.input_path} ({row.input_hash[:20]}...)")
print(f"Output: {row.output_path} ({row.output_hash[:20]}...)")

# Missing job returns None
assert proc_storage.get_by_job_id("nonexistent") is None
print("get_by_job_id returns None for missing job: OK")

Retrieved: job_conv_001 (convert)
Input: /tmp/source.mkv (sha256:aaaaaaaaaaaaa...)
Output: /tmp/output.mp4 (sha256:bbbbbbbbbbbbb...)
get_by_job_id returns None for missing job: OK


In [ ]:
# Save an extract_segment job and test list_jobs
proc_storage.save(
    job_id="job_ext_001",
    action="extract_segment",
    input_path="/tmp/source.mkv",
    input_hash="sha256:" + "a" * 64,
    output_path="/tmp/segment_10-20.wav",
    output_hash="sha256:" + "c" * 64,
    parameters={"start": 10.0, "end": 20.0}
)

jobs = proc_storage.list_jobs()
assert len(jobs) == 2
assert jobs[0].job_id == "job_ext_001"  # Newest first
assert jobs[0].action == "extract_segment"

print(f"list_jobs returned {len(jobs)} rows: {[(j.job_id, j.action) for j in jobs]}")

list_jobs returned 2 rows: [('job_ext_001', 'extract_segment'), ('job_conv_001', 'convert')]


In [ ]:
# Cleanup
os.unlink(tmp_db2.name)
print("Processing storage cleanup complete")

Processing storage cleanup complete


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()